In [ ]:
## try out different gpt models for trulens evaluation
### ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo')
### ChatOpenAI(temperature=0.9,model_name='gpt-3.5-turbo')
### ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo-16k')
### ChatOpenAI(temperature=0.9,model_name='gpt-3.5-turbo-16k')
### ChatOpenAI(temperature=1.0,model_name='gpt-3.5-turbo-16k')

In [ ]:
! pip install streamlit streamlit_chat langchain openai faiss-cpu tiktoken

## **Create Vector Stores**

In [ ]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<enter key>"
os.environ["HUGGINGFACE_API_KEY"] = "<enter key>"

### Retriever 1 - NPS Scores

In [4]:
file_path = "nps_scores.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [6]:
retriever_1 = vectorstore.as_retriever()

### Retriever 2 - Overall Ratings

In [7]:
file_path = "overall.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [8]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [9]:
retriever_2 = vectorstore.as_retriever()

### Retriever 3 - Interest Rates

In [10]:
file_path = "interest-rates.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [11]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [12]:
retriever_3 = vectorstore.as_retriever()

### Retriever 4 - Cash Back

In [13]:
file_path = "cash-back.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [14]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [15]:
retriever_4 = vectorstore.as_retriever()

### Retriever 5 - Overdraft

In [16]:
file_path = "overdraft.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [17]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [18]:
retriever_5 = vectorstore.as_retriever()

### Retriever 6 - Packaged Accounts

In [19]:
file_path = "packaged.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [20]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [21]:
retriever_6 = vectorstore.as_retriever()

### Retriever 7 - Basic Accounts

In [22]:
file_path = "basic-accounts.csv"
loader = CSVLoader(file_path=file_path, encoding="utf-8", csv_args={
                'delimiter': ','})
data = loader.load()

In [23]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)

In [24]:
retriever_7 = vectorstore.as_retriever()

## **Build Chatbot using Agent**

In [25]:
from langchain.agents.agent_toolkits import create_retriever_tool, create_conversational_retrieval_agent

In [26]:
tool_1 = create_retriever_tool(
    retriever_1,
    "search_nps_scores",
    "Searches and returns nps scores for banks to provide user with information about customer satisfaction.",
)
tool_2 = create_retriever_tool(
    retriever_2,
    "search_overall_bank_ratings",
    "Searches the ratings of banks across different criteria to answer the user's questions about the banks or provide the requested advice about which bank suits their needs"
)
tool_3 = create_retriever_tool(
    retriever_3,
    "search_interest_rates",
    "Searches information about interest rates offered on banks' account and customer ratings for those banks to provide advice on the suitability of respective bank for user"
)
tool_4 = create_retriever_tool(
    retriever_4,
    "search_cash_back",
    "Searches information about cashback offered on banks' account and customer ratings for those banks to provide advice on the suitability of respective bank for user"
)
tool_5 = create_retriever_tool(
    retriever_5,
    "search_overdraft",
    "Searches information about overdraft fees charged by banks' account and customer ratings for those banks to provide advice on the suitability of respective bank for user"
)
tool_6 = create_retriever_tool(
    retriever_6,
    "search_packaged_accounts",
    "Searches information about packaged accounts offered by banks and customer ratings for those banks to provide advice on the suitability of respective bank for user"
)
tool_7 = create_retriever_tool(
    retriever_7,
    "search_basic_accounts",
    "Searches information about basic accounts offered by banks and customer ratings for those banks to provide advice on the suitability of respective bank for user"
)

In [27]:
tools = [tool_1, tool_3, tool_4, tool_5, tool_6, tool_7, tool_2]

In [28]:
agent_executor = create_conversational_retrieval_agent(llm=ChatOpenAI(temperature=1.0,model_name='gpt-3.5-turbo-16k'), tools=tools, verbose=True)

## **TruLens Dashboard**

In [ ]:
! pip install trulens_eval==0.17.0

In [30]:
# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
from trulens_eval.schema import FeedbackResult
tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [31]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

✅ In language_match, input text1 will be set to __record__.main_input or `Select.RecordInput` .
✅ In language_match, input text2 will be set to __record__.main_output or `Select.RecordOutput` .


In [32]:
tru_recorder = TruChain(agent_executor,
    app_id='Which_Bank_1.0_GPT3_16k',
    feedbacks=[f_lang_match])

✅ added app Which_Bank_1.0_GPT3_16k
✅ added feedback definition feedback_definition_hash_ae62ec19954d41bed0cd52b9c252ab4a


In [33]:
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 4.109s

Go to this url and submit the ip given here. your url is: https://chubby-paths-exist.loca.lt

  Submit this IP Address: 35.231.33.115



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
! curl ipv4.icanhazip.com

34.74.175.64


In [ ]:
tru.stop_dashboard()

## **Prompts**

In [34]:
prompt_2 = "Which bank had the lowest NPS score in 2019?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_2)
llm_response["output"]



> Entering new AgentExecutor chain...

Invoking: `search_nps_scores` with `{'year': '2019'}`


[Document(page_content='\ufeffBank Name: HSBC\nPercentage score: 59', metadata={'source': 'nps_scores.csv', 'row': 10}), Document(page_content='\ufeffBank Name: Monzo Bank\nPercentage score: 86', metadata={'source': 'nps_scores.csv', 'row': 0}), Document(page_content='\ufeffBank Name: Starling Bank\nPercentage score: 84', metadata={'source': 'nps_scores.csv', 'row': 1}), Document(page_content='\ufeffBank Name: Santander\nPercentage score: 61', metadata={'source': 'nps_scores.csv', 'row': 7})]The bank with the lowest NPS score in 2019 was HSBC with a score of 59%.

> Finished chain.
✅ added record record_hash_75790ca06b8723b9a315e67446999e4c


'The bank with the lowest NPS score in 2019 was HSBC with a score of 59%.'

In [35]:
prompt_3 = "What bank has the best interest rate for savings accounts?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_3)
llm_response["output"]



> Entering new AgentExecutor chain...

Invoking: `search_interest_rates` with `{'account_type': 'savings'}`


[Document(page_content='\ufeffBank and Account Name: Nationwide Building Society FlexDirect\nCustomer Score: 78%\nMaximum interest rate: 5%\nMax deposit earning interest: £1,500.00\nMin funding/month: £1,000.00', metadata={'source': 'interest-rates.csv', 'row': 3}), Document(page_content='\ufeffBank and Account Name: Santander Edge Up\nCustomer Score: 70%\nMaximum interest rate: 3.50%\nMax deposit earning interest: £25,000.00\nMin funding/month: £1,500.00', metadata={'source': 'interest-rates.csv', 'row': 2}), Document(page_content='\ufeffBank and Account Name: Bank of Scotland Classic with Vantage\nCustomer Score: 72%\nMaximum interest rate: 3%\nMax deposit earning interest: £5,000.00\nMin funding/month: £1,000.00', metadata={'source': 'interest-rates.csv', 'row': 6}), Document(page_content='\ufeffBank and Account Name: Starling Current Account\nCustomer Score: 83%\nMaximum 

'According to the available information, the bank with the best interest rate for savings accounts is Nationwide Building Society FlexDirect. It offers a maximum interest rate of 5%.'

In [36]:
prompt_4 = "What are the top 3 banks by NPS scores?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_4)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_cf555fcc26cde1047660dd1cf5d6997f

Invoking: `search_nps_scores` with `{'year': '2021'}`


[Document(page_content='\ufeffBank Name: Santander\nPercentage score: 61', metadata={'source': 'nps_scores.csv', 'row': 7}), Document(page_content='\ufeffBank Name: Nationwide\nPercentage score: 71', metadata={'source': 'nps_scores.csv', 'row': 4}), Document(page_content='\ufeffBank Name: TSB\nPercentage score: 51', metadata={'source': 'nps_scores.csv', 'row': 15}), Document(page_content='\ufeffBank Name: Metro Bank\nPercentage score: 81', metadata={'source': 'nps_scores.csv', 'row': 3})]The top 3 banks by NPS scores are:

1. Metro Bank with a score of 81%
2. Nationwide with a score of 71%
3. Santander with a score of 61%

> Finished chain.
✅ added record record_hash_9cd99d0adc39eac9a69ed626ecca3faa


'The top 3 banks by NPS scores are:\n\n1. Metro Bank with a score of 81%\n2. Nationwide with a score of 71%\n3. Santander with a score of 61%'

In [37]:
prompt_5 = "which bank has five stars for service"
# record response
with tru_recorder as recording:
  llm_response = agent_executor(prompt_5)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_6007c0ed07f568abb32c3d7e845d7d72

Invoking: `search_overall_bank_ratings` with `{'criteria': 'service'}`


[Document(page_content='\ufeffBank Name: Revolut\nCustomer Score: 76%\nCustomer service in general: 3 Star\nApplication process: 5 Star\nComplaints and Resolutions: 3 Star\nService in branch: -\nTelephone banking service: -\nOnline banking service: 4 Star\nMobile banking app: 4 Star\nCalling customer services: -', metadata={'source': 'overall.csv', 'row': 6}), Document(page_content='\ufeffBank Name: Santander\nCustomer Score: 70%\nCustomer service in general: 4 Star\nApplication process: -\nComplaints and Resolutions: 3 Star\nService in branch: 3 Star\nTelephone banking service: 3 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 3 Star', metadata={'source': 'overall.csv', 'row': 16}), Document(page_content='\ufeffBank Name: Chase\nCustomer 

'The bank that has five stars for service is Chase.'

In [38]:
prompt_6 = "which bank is best for international students?"
# record response
with tru_recorder as recording:
  llm_response = agent_executor(prompt_6)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_4caf5b45240c340db9344d1cbdf72003

Invoking: `search_overall_bank_ratings` with `{'criteria': 'international students'}`


[Document(page_content='\ufeffBank Name: Bank of Ireland UK\nCustomer Score: 63%\nCustomer service in general: 3 Star\nApplication process: -\nComplaints and Resolutions: 3 Star\nService in branch: 3 Star\nTelephone banking service: 3 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 3 Star', metadata={'source': 'overall.csv', 'row': 20}), Document(page_content='\ufeffBank Name: HSBC\nCustomer Score: 69%\nCustomer service in general: 3 Star\nApplication process: 4 Star\nComplaints and Resolutions: 3 Star\nService in branch: 2 Star\nTelephone banking service: 2 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 2 Star', metadata={'source': 'overall.csv', 'row': 17}), Document(page_conte

'Based on the available information, Barclays Bank is considered the best bank for international students. It has a customer score of 74% and receives positive ratings for customer service, application process, service in branch, and mobile banking app.'

In [39]:
prompt_7 = "Can you explain why?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_7)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_ba353169726ee1600b276ec25660249a

Invoking: `search_overall_bank_ratings` with `{'criteria': 'international students'}`


[Document(page_content='\ufeffBank Name: Bank of Ireland UK\nCustomer Score: 63%\nCustomer service in general: 3 Star\nApplication process: -\nComplaints and Resolutions: 3 Star\nService in branch: 3 Star\nTelephone banking service: 3 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 3 Star', metadata={'source': 'overall.csv', 'row': 20}), Document(page_content='\ufeffBank Name: HSBC\nCustomer Score: 69%\nCustomer service in general: 3 Star\nApplication process: 4 Star\nComplaints and Resolutions: 3 Star\nService in branch: 2 Star\nTelephone banking service: 2 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 2 Star', metadata={'source': 'overall.csv', 'row': 17}), Document(page_conte

'Barclays Bank is considered the best bank for international students based on several factors. \n\nFirstly, it has a customer score of 74%, indicating that customers have generally had a positive experience with the bank. \n\nSecondly, Barclays Bank receives positive ratings for customer service. This is important for international students who may have specific needs or questions regarding their banking services while studying abroad. \n\nAdditionally, Barclays Bank scores well in terms of service in branch and online banking service, which are crucial for international students who may need access to banking services both physically and remotely. \n\nFurthermore, Barclays Bank has a highly rated mobile banking app, which can be convenient for international students who rely on their smartphones for convenient and easy access to banking services.\n\nOverall, these factors contribute to Barclays Bank being considered the best bank for international students.'

In [40]:
prompt_8 = "What is the best bank for international transactions?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_8)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_fe8d1bfc4e48a8141b5a42ceab35266a

Invoking: `search_overall_bank_ratings` with `{'criteria': 'international transactions'}`


[Document(page_content='\ufeffBank Name: Bank of Ireland UK\nCustomer Score: 63%\nCustomer service in general: 3 Star\nApplication process: -\nComplaints and Resolutions: 3 Star\nService in branch: 3 Star\nTelephone banking service: 3 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 3 Star', metadata={'source': 'overall.csv', 'row': 20}), Document(page_content='\ufeffBank Name: HSBC\nCustomer Score: 69%\nCustomer service in general: 3 Star\nApplication process: 4 Star\nComplaints and Resolutions: 3 Star\nService in branch: 2 Star\nTelephone banking service: 2 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 2 Star', metadata={'source': 'overall.csv', 'row': 17}), Document(page_c

'Based on the available information, Revolut is considered the best bank for international transactions. It has a customer score of 76% and receives positive ratings for application process, online banking service, and mobile banking app. Revolut specializes in international transactions and offers features such as multi-currency accounts, low fees for foreign exchange transactions, and seamless transfers between different currencies. These features make it convenient and cost-effective for individuals who frequently make international transactions.'

In [41]:
prompt_8_2 = "Can you tell me which of these offers low international transaction fees?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_8_2)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_db3e2f465297d430c589793867559a2b

Invoking: `search_overall_bank_ratings` with `{'criteria': 'international transaction fees'}`


[Document(page_content='\ufeffBank Name: Bank of Ireland UK\nCustomer Score: 63%\nCustomer service in general: 3 Star\nApplication process: -\nComplaints and Resolutions: 3 Star\nService in branch: 3 Star\nTelephone banking service: 3 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 3 Star', metadata={'source': 'overall.csv', 'row': 20}), Document(page_content='\ufeffBank Name: Revolut\nCustomer Score: 76%\nCustomer service in general: 3 Star\nApplication process: 5 Star\nComplaints and Resolutions: 3 Star\nService in branch: -\nTelephone banking service: -\nOnline banking service: 4 Star\nMobile banking app: 4 Star\nCalling customer services: -', metadata={'source': 'overall.csv', 'row': 6}), Document(page_content='\

'Among the available options, Revolut is known for offering low international transaction fees. It has a customer score of 76% and receives positive ratings for online banking service and mobile banking app. Revolut specializes in international transactions and aims to provide competitive exchange rates and low fees for foreign exchange transactions. This makes it a popular choice for individuals who frequently make international transactions and want to save on fees.'

In [42]:
prompt_9 = "which bank has the best cashback offers?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_9)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_0bf02de3690805fd90b1833f37d2fb61

Invoking: `search_cash_back` with `{'bank': 'all'}`


[Document(page_content='\ufeffBank Name: \nCustomer score: \nMax cashback rate/month: \nMonthly fee: \nMin funding/month: ', metadata={'source': 'cash-back.csv', 'row': 10}), Document(page_content="\ufeffBank Name: Halifax 'Reward Extras'\nCustomer score: 74%\nMax cashback rate/month: £5 or monthly reward benefit eg free Vue ticket\nMonthly fee: £3 (free if you pay in £1,500/mth)\nMin funding/month: £1,500.00", metadata={'source': 'cash-back.csv', 'row': 6}), Document(page_content="\ufeffBank Name: Santander 'Edge Up'\nCustomer score: 70%\nMax cashback rate/month: £30.00\nMonthly fee: £5.00\nMin funding/month: £1,500.00", metadata={'source': 'cash-back.csv', 'row': 0}), Document(page_content="\ufeffBank Name: TSB 'Spend & Save Plus'\nCustomer score: 69%\nMax cashback rate/month: £5.00\nMonthly fee: £3.

"Among the available options, there are several banks that offer cashback rewards. Here are some of the banks and their cashback offers:\n\n1. Halifax 'Reward Extras':\n   - Customer Score: 74%\n   - Max cashback rate/month: £5 or monthly reward benefit (e.g., free Vue ticket)\n   - Monthly fee: £3 (free if you pay in £1,500/mth)\n   - Min funding/month: £1,500.00\n\n2. Santander 'Edge Up':\n   - Customer Score: 70%\n   - Max cashback rate/month: £30.00\n   - Monthly fee: £5.00\n   - Min funding/month: £1,500.00\n\n3. TSB 'Spend & Save Plus':\n   - Customer Score: 69%\n   - Max cashback rate/month: £5.00\n   - Monthly fee: £3.00\n   - Min funding/month: £0.00\n\nThese banks offer cashback rewards based on specific criteria, such as meeting minimum funding requirements or paying certain fees. It's important to consider your own spending habits and requirements when choosing a bank with the best cashback offers."

In [43]:
prompt_10 = "Which is the least recommended back overall?"
with tru_recorder as recording:
  llm_response = agent_executor(prompt_10)
llm_response["output"]



> Entering new AgentExecutor chain...
✅ feedback result language_match DONE feedback_result_hash_dfeb5fd0f7dd938573b969fccec4dbe8

Invoking: `search_overall_bank_ratings` with `{'criteria': 'overall'}`


[Document(page_content='\ufeffBank Name: Revolut\nCustomer Score: 76%\nCustomer service in general: 3 Star\nApplication process: 5 Star\nComplaints and Resolutions: 3 Star\nService in branch: -\nTelephone banking service: -\nOnline banking service: 4 Star\nMobile banking app: 4 Star\nCalling customer services: -', metadata={'source': 'overall.csv', 'row': 6}), Document(page_content='\ufeffBank Name: HSBC\nCustomer Score: 69%\nCustomer service in general: 3 Star\nApplication process: 4 Star\nComplaints and Resolutions: 3 Star\nService in branch: 2 Star\nTelephone banking service: 2 Star\nOnline banking service: 3 Star\nMobile banking app: 3 Star\nCalling customer services: 2 Star', metadata={'source': 'overall.csv', 'row': 17}), Document(page_content='\ufeffBank Name: Halifax\nCustome

'Based on the available information, the least recommended bank overall is HSBC. It has a customer score of 69% and receives lower ratings in various categories, including customer service in general, service in branch, telephone banking service, and calling customer services. While HSBC has positive ratings for the application process and online banking service, its overall score and lower ratings in important areas contribute to it being considered the least recommended bank overall.'